##Este codigo transforma la lista de DYD en lista Royal Minorista.

## ATENCION: antes de subir la lista de DYD, eliminarle la fecha al nombre, de modo que su nombre quede solo como "Mayorista"

Completar "porcentaje de ganancia" y "compra minima": 











In [1]:
# Sacar precios de Emprende Tienda

precio_ON_2lbs = 10600
precio_ON_5lbs = 19900
precio_Syntha6 = 14900
precio_Edge = 10790

In [4]:
# Instalacion
!pip install tika
import pandas as pd
import numpy as np

# Correccion
from tika import parser
raw = parser.from_file('preferencial.pdf')
df = (raw['content'])
df = df.replace('\n',',')
df = df.replace(',,',',')
df = df.replace(',,,,,,,,,,,,,,,,,,,,','')

# Reemplazo de las comas en los precios (para luego dividir los productos por coma)
for i,v in enumerate(df):
  if i < len(df)-3:
    if v == ',':
      if df[i+2] == '0':
        df = df[:i] + '.' + df[i + 1:]

      try:
        valor1 = int(df[i+1])
        if type(valor1) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
        continue

      try:
        valor2 = int(df[i+2])
        if type(valor2) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
          continue


# Split
df = df.split(',')

# lista de precios.
total_precios = []
for i1,v1 in enumerate(df):
  for i2,v2 in enumerate(v1):
    if v2 == '$':
      total_precios.append(int(str(round(int(df[i1][i2+2:])))))

# Cuartiles de precios
df_precios = pd.DataFrame(total_precios).describe().reset_index()
q1 = df_precios.query("index == '25%'").iloc[0,1]
q2 = df_precios.query("index == '50%'").iloc[0,1]
q3 = df_precios.query("index == '75%'").iloc[0,1]
q4 = df_precios.query("index == 'max'").iloc[0,1]

# Agregar ganancia
df1 = df.copy()
for i1,v1 in enumerate(df1):
  for i2,v2 in enumerate(v1):
    if v2 == '$':
      size = len(df1[i1])
      to_delete = len(df1[i1][i2+2:])
      mod_string = df1[i1][:size - to_delete]
      old_price = (round(int(df1[i1][i2+2:])))
      multiplicador = 0
      porcentaje_ganancia = 0

      # Ganancia segun decil
      ganancias = [0.50, 0.53, 0.56, 0.59, 0.62, 0.65, 0.69, 0.72, 0.75, 0.8]
      percentiles_precios = np.percentile(total_precios, np.arange(0, 100, 10)).tolist()
      porcentaje_ganancia = 0
      for i in range(len(percentiles_precios)):
        try:
          if old_price in range(int(percentiles_precios[i]), int(percentiles_precios[i+1])):
            porcentaje_ganancia = ganancias[i]
            break
        except:
          if old_price >= percentiles_precios[-1]:
            porcentaje_ganancia = ganancias[-1]
            break
          else:
            print(f'Precio {old_price} no se ubica en percentiles')
            break

      mod_string += str(round(old_price / porcentaje_ganancia))
      df1[i1] = mod_string
df = df1.copy()

# Agrego la ON y la BSN
precio_ON_2lbs = f'Optimun Nutrition - 100% Whey Gold Vainilla Icecream X 2 lbs $ {precio_ON_2lbs}'
precio_ON_5lbs = f'Optimun Nutrition - 100% Whey Gold Vainilla Icecream X 5 lbs $ {precio_ON_5lbs}'
precio_Syntha6 = f'BSN - Syntha 6 Edge X 4 lbs Vainilla $ {precio_Syntha6}'
precio_Edge = f'BSN - Syntha 6 Isolate X 2 lbs $ {precio_Edge}'
df2 = df1.copy()
for i1,v1 in enumerate(df1):
  if precio_ON_2lbs not in df2:
    if v1[0:7] == 'Optimun':
      df2.insert(i1+2, precio_ON_2lbs)
      df2.insert(i1+2, precio_ON_5lbs)
  if precio_Syntha6 not in df2:  
    if v1[0:3] == 'BSN':
      df2.insert(i1-1, precio_Syntha6)
      df2.insert(i1-1, precio_Edge)

df = df2

# Agrego la ON y la BSN
precio_ON_2lbs = f'Optimun Nutrition - 100% Whey Gold Vainilla Icecream X 2 lbs $ {precio_ON_2lbs}'
precio_Syntha6 = f'BSN - Syntha 6 Edge X 4 lbs Vainilla $ {precio_Syntha6}'
precio_Edge = f'BSN - Syntha 6 Isolate X 2 lbs $ {precio_Edge}'
df2 = df1.copy()
for i1,v1 in enumerate(df1):
  if precio_ON_2lbs not in df2:
    if v1[0:7] == 'Optimun':
      df2.insert(i1+2, precio_ON_2lbs)
  if precio_Syntha6 not in df2:  
    if v1[0:3] == 'BSN':
      df2.insert(i1-1, precio_Syntha6)
      df2.insert(i1-1, precio_Edge)

df = df2

# Quitar filas irrelevantes
for i,v in enumerate(df):
  if v.startswith('Fecha') or v.startswith('Página') or v.startswith('MARCA') or v.startswith('Lista'):
     df[i] = ''

# Eliminado de espacios vacios
while '' in df:
  df.remove('')

new_df = []
for v in df:
  if '$' in v:
    new_df.append(v)
df = new_df.copy()

# Encabezado piola
df.insert(0, 'ROYAL DISTRIBUIDORA \n \n  LISTA MINORISTA \n \n \n MARCA    CODIGO       PRODUCTO                  PRECIO\n \n')

# Formato prolijo
n = '\n'.join(df)

# Guardado
text_file = open("Lista Minorista.txt", "w")
text_file.write(n)
text_file.close()
from google.colab import files
files.download('Lista Minorista.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>